In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import json
import time
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\kheat\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
# URL of page to be scraped
url = 'https://kenpom.com/index.php'

browser.visit(url)
time.sleep(0.5)

html = browser.html
soup = bs(html, "lxml")

In [4]:
dfs = pd.read_html(browser.html)
df = dfs[0]

df.columns = [x[1] for x in df.columns]
df

,Rk,Team,Conf,W-L,AdjEM,AdjO,AdjO,AdjD,AdjD,AdjT,...,Luck,Luck,AdjEM,AdjEM,OppO,OppO,OppD,OppD,AdjEM,AdjEM
0,1,Gonzaga 1,WCC,26-0,36.59,126.0,1,89.4,9,74.7,...,0.018,136,7.93,96,105.9,99,98.0,97,6.34,106
1,2,Illinois 1,B10,24-6,30.70,118.8,8,88.1,4,70.6,...,0.026,117,18.27,5,112.4,4,94.1,11,9.99,62
2,3,Michigan 1,B10,20-4,30.50,119.3,7,88.8,7,66.8,...,0.026,118,16.75,15,111.3,11,94.6,20,2.73,166
3,4,Baylor 1,B12,23-2,29.93,123.0,3,93.0,34,68.7,...,0.047,78,9.64,81,106.3,91,96.7,67,-3.71,267
4,5,Houston 2,Amer,25-3,29.68,119.6,6,89.9,13,65.0,...,-0.011,206,6.82,106,103.9,152,97.1,75,1.99,184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,353,South Carolina St.,MEAC,1-17,-25.02,87.1,351,112.1,337,71.9,...,-0.109,346,-4.54,277,98.2,310,102.7,205,2.12,180
353,354,Alabama St.,SWAC,4-14,-26.12,84.2,355,110.3,321,71.0,...,-0.049,284,-14.86,347,91.2,347,106.1,338,-99.00,331
354,355,Maryland Eastern Shore,MEAC,0-0,-27.33,83.9,356,111.3,333,68.0,...,0.000,172,-99.00,348,0.0,348,200.0,348,-99.00,331
355,356,Chicago St.,WAC,0-9,-29.52,88.3,348,117.8,356,67.7,...,-0.006,197,9.28,84,106.8,83,97.6,86,9.28,65


In [5]:
df.Team[0][-2:]

' 1'

In [6]:
str.isnumeric(df.Team[0][-2:].strip())

True

In [7]:
clean_names = []
seeds = []

for _, row in df.iterrows():
    seed = row.Team[-2:].strip()
    
    if str.isnumeric(seed):
        clean_name = row.Team[:-2]
        
    else:
        clean_name = row.Team
        seed = ''
        
    clean_names.append(clean_name)
    seeds.append(seed)
  

df["TeamName_Clean"] = clean_names
df["Seed"] = seeds

df.head(30)

,Rk,Team,Conf,W-L,AdjEM,AdjO,AdjO,AdjD,AdjD,AdjT,...,AdjEM,AdjEM,OppO,OppO,OppD,OppD,AdjEM,AdjEM,TeamName_Clean,Seed
0,1,Gonzaga 1,WCC,26-0,36.59,126.0,1,89.4,9,74.7,...,7.93,96,105.9,99,98.0,97,6.34,106,Gonzaga,1
1,2,Illinois 1,B10,24-6,30.70,118.8,8,88.1,4,70.6,...,18.27,5,112.4,4,94.1,11,9.99,62,Illinois,1
2,3,Michigan 1,B10,20-4,30.50,119.3,7,88.8,7,66.8,...,16.75,15,111.3,11,94.6,20,2.73,166,Michigan,1
3,4,Baylor 1,B12,23-2,29.93,123.0,3,93.0,34,68.7,...,9.64,81,106.3,91,96.7,67,-3.71,267,Baylor,1
4,5,Houston 2,Amer,25-3,29.68,119.6,6,89.9,13,65.0,...,6.82,106,103.9,152,97.1,75,1.99,184,Houston,2
5,6,Iowa 2,B10,21-8,28.72,123.5,2,94.7,52,69.9,...,16.85,14,110.4,21,93.5,6,-5.13,287,Iowa,2
6,7,Alabama 2,SEC,24-6,24.87,111.7,35,86.9,2,73.8,...,14.25,32,109.7,32,95.5,35,9.15,67,Alabama,2
7,8,Loyola Chicago 8,MVC,25-4,24.63,111.5,37,86.9,3,64.0,...,5.16,121,104.8,126,99.7,129,5.54,121,Loyola Chicago,8
8,9,Ohio St. 2,B10,21-10,24.19,120.8,4,96.6,73,67.0,...,18.19,6,111.7,7,93.5,5,2.84,164,Ohio St.,2
9,10,Wisconsin 9,B10,18-12,23.59,113.5,27,89.9,12,64.8,...,18.12,7,111.7,8,93.6,7,1.39,195,Wisconsin,9


In [46]:
df.columns = ['Rk', 'Team', 'Conf', 'W-L', 'AdjEM', 'AdjO', 'AdjO_Rank', 'AdjD', 'AdjD_Rank',
       'AdjT', 'AdjT_Rank', 'Luck', 'Luck_Rank', 'SOS_AdjEM', 'AdjEM_Rank', 'SOS_OppO', 'OppO_Rank',
       'SOS_OppD', 'OppD_Rank', 'NCSOS_AdjEM', 'AdjEM_Rank', 'TeamName_Clean', 'Seed']

df

,Rk,Team,Conf,W-L,AdjEM,AdjO,AdjO_Rank,AdjD,AdjD_Rank,AdjT,...,SOS_AdjEM,AdjEM_Rank,SOS_OppO,OppO_Rank,SOS_OppD,OppD_Rank,NCSOS_AdjEM,AdjEM_Rank,TeamName_Clean,Seed
0,1,Gonzaga 1,WCC,26-0,36.59,126.0,1,89.4,9,74.7,...,7.93,96,105.9,99,98.0,97,6.34,106,Gonzaga,1
1,2,Illinois 1,B10,24-6,30.70,118.8,8,88.1,4,70.6,...,18.27,5,112.4,4,94.1,11,9.99,62,Illinois,1
2,3,Michigan 1,B10,20-4,30.50,119.3,7,88.8,7,66.8,...,16.75,15,111.3,11,94.6,20,2.73,166,Michigan,1
3,4,Baylor 1,B12,23-2,29.93,123.0,3,93.0,34,68.7,...,9.64,81,106.3,91,96.7,67,-3.71,267,Baylor,1
4,5,Houston 2,Amer,25-3,29.68,119.6,6,89.9,13,65.0,...,6.82,106,103.9,152,97.1,75,1.99,184,Houston,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,353,South Carolina St.,MEAC,1-17,-25.02,87.1,351,112.1,337,71.9,...,-4.54,277,98.2,310,102.7,205,2.12,180,South Carolina St.,
353,354,Alabama St.,SWAC,4-14,-26.12,84.2,355,110.3,321,71.0,...,-14.86,347,91.2,347,106.1,338,-99.00,331,Alabama St.,
354,355,Maryland Eastern Shore,MEAC,0-0,-27.33,83.9,356,111.3,333,68.0,...,-99.00,348,0.0,348,200.0,348,-99.00,331,Maryland Eastern Shore,
355,356,Chicago St.,WAC,0-9,-29.52,88.3,348,117.8,356,67.7,...,9.28,84,106.8,83,97.6,86,9.28,65,Chicago St.,


In [49]:
df_kenPom = df.drop(['Team','W-L','AdjO_Rank', 'AdjD_Rank', 'AdjT_Rank','Luck_Rank','AdjEM_Rank', 'OppO_Rank', 'OppD_Rank','AdjEM_Rank', 'Seed'], axis=1)

In [52]:
df_kenPom = df_kenPom[['Rk', 'TeamName_Clean', 'AdjEM', 'AdjO', 'AdjD', 'AdjT', 'Luck', 'SOS_AdjEM', 'SOS_OppO', 'SOS_OppD', 'NCSOS_AdjEM']]

In [56]:
df_kenPom.to_csv('Data/2021/kenPom.csv', index=False)